In [1]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense, StringLookup
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from keras.src.layers import LSTM

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products_10k.pkl')
pandas_interaction_data = pandas_interaction_data.sample(9000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)
product_data['color_mapped'] = pd.factorize(product_data['productColorName'])[0]

pandas_interaction_data = pd.merge(pandas_interaction_data, product_data[['main_product_id', 'color_mapped', 'productName']],
                                   left_on='main_product_id',
                                   right_on='main_product_id',
                                    how='left')

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount,color_mapped,productName
0,5474118,7699535,1,34,Sneaker low Materialmix Leder lila
1,2176279,3845441,1,4,Sneaker low Materialmix Leder grau
2,2230073,3591304,1,4,Sneaker low Rauleder grau
3,9195905,4821963,1,3,Pantolette Glattleder braun
4,9111516,3591163,1,0,Sneaker high Glattleder schwarz
...,...,...,...,...,...
8995,9354273,548300,1,0,Sportliche Ballerina Materialmix Leder/Lederim...
8996,5592224,3477220,1,1,Sportliche Ballerina Rauleder blau
8997,6932845,3693122,1,0,Schnürstiefelette Rauleder schwarz
8998,10532624,4773825,1,6,Sneaker low Glattleder beige


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40....",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]",30
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]",56
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[],1
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0],33


In [4]:
# Initialize tokenizer
string_lookup = StringLookup()
string_lookup.adapt(pandas_interaction_data['productName'])

tokenizer = Tokenizer(num_words=string_lookup.vocabulary_size(), oov_token="<OOV>")
tokenizer.fit_on_texts(pandas_interaction_data['productName'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(pandas_interaction_data['productName'])

# Pad sequences
padded_sequences = pad_sequences(sequences, padding='post')

In [5]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values, 
    'product_color': pandas_interaction_data['color_mapped'].values, 
    'product_description': padded_sequences, 
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [6]:
interaction_dataset

<_TensorSliceDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(8,), dtype=tf.int32, name=None)}>

In [7]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(8,), dtype=tf.int32, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(8,), dtype=tf.int32, name=None)}>

In [8]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

unique_colors = np.array(product_data["color_mapped"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)

product_colors_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_colors_vocabulary.adapt(unique_colors)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [9]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, unique_product_color_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])

        # Color embeddings
        self.color_embedding = tf.keras.Sequential([
            unique_product_color_vocab,
            Embedding(unique_product_color_vocab.vocabulary_size(), embedding_dimension)
        ]) 


        # Textual description model
        self.textual_description_model = tf.keras.Sequential([
            Embedding(input_dim=unique_product_ids_vocab.vocabulary_size(), output_dim=embedding_dimension),
            LSTM(64),
            Dense(64, activation='relu')
        ])

        # New embeddings for color and description
        self.reduced_product_embeddings = Dense(64, activation='relu')
        self.reduced_description_embeddings = Dense(32, activation='relu')
        self.reduced_color_embeddings = Dense(32, activation='relu')
        
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, input_shape=(256,), activation="relu")

        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])

        color_embeddings = self.color_embedding(features["product_color"])
        description_embeddings = self.textual_description_model(features["product_description"])

        reduced_product_embeddings = self.reduced_product_embeddings(product_embeddings)
        reduced_description_embeddings = self.reduced_description_embeddings(description_embeddings)
        reduced_color_embeddings = self.reduced_color_embeddings(color_embeddings)


        # Now concatenate
        user_combined = tf.concat([user_embeddings, ], axis=1)  # shape will be [None, 128]
        product_combined = tf.concat([reduced_product_embeddings, reduced_color_embeddings, reduced_description_embeddings], axis=1)  # shape will be [None, 128]
        


        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_combined)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [10]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, product_colors_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [11]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3) 
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
57/57 [==============================] - 22s 353ms/step - auc: 0.0000e+00 - rmse: 0.9890 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.1835 - factorized_top_k/top_5_categorical_accuracy: 0.2511 - factorized_top_k/top_10_categorical_accuracy: 0.2824 - factorized_top_k/top_50_categorical_accuracy: 0.3757 - factorized_top_k/top_100_categorical_accuracy: 0.4325 - loss: 603.4776 - regularization_loss: 0.0000e+00 - total_loss: 603.4776
Epoch 2/3
57/57 [==============================] - 22s 388ms/step - auc: 0.0000e+00 - rmse: 2.0142 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9592 - factorized_top_k/top_5_categorical_accuracy: 0.9696 - factorized_top_k/top_10_categorical_accuracy: 0.9718 - factorized_top_k/top_50_categorical_accuracy: 0.9814 - factorized_top_k/top_100_categorical_accuracy: 0.9844 - loss: 581.8840 - regularization_loss: 0.0000e+00 - total_loss: 581.8840
Epoch 3/3
57/57 

In [12]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

57/57 [==============================] - 27s 469ms/step - auc: 0.0000e+00 - rmse: 28.7465 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 229.4055 - regularization_loss: 0.0000e+00 - total_loss: 229.4055


([0.0,
  28.74653434753418,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  34.86231231689453,
  0,
  34.86231231689453],
 [1.0, 0.9811111, 0.9904655364261746])

In [13]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

29/29 [==============================] - 16s 543ms/step - auc: 0.0000e+00 - rmse: 18.9973 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 344.2931 - regularization_loss: 0.0000e+00 - total_loss: 344.2931


([0.0,
  18.997344970703125,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  19.998382568359375,
  0,
  19.998382568359375],
 [1.0, 0.99333334, 0.9966555215948624])

In [14]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		18.997344970703125
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		1.0
factorized_top_k/top_5_categorical_accuracy		1.0
factorized_top_k/top_10_categorical_accuracy		1.0
factorized_top_k/top_50_categorical_accuracy		1.0
factorized_top_k/top_100_categorical_accuracy		1.0
AUC: 0.0


In [15]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 7779418,  5801042,  7174137,  8803464,  5011771,  4344821,
        7231455,  7298386,  7301293,  1955643,  6804279, 10910614,
        9608740, 10420444,  7998675,  6914415,  9961496,  6610340,
        1991347, 11073419,  6919761,  5859676,  2189096,  3872363,
        2144655,  4721289,  9718476,  5354392,  2027792, 10538803,
        2045509,  9407468,  2161753,  1949252,  5182610,  5392016,
        9095687,  8906933,  2127575,  9832965,  7553256,  7735689,
        8123061, 10622837,  5770566,  5359761,  2535983,  2057175,
        9277280,  6948602, 10871119,  2070452,  7410809,  2104353,
        8663095,  2037250,  7646877,  3812866,  7248404,  4803790,
        9812408, 10211791,  5511081, 10143601])

In [16]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  7779418
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
59,3477019,4060666971607,Sneaker low Materialmix Leder blau,blau,56.435.36,6590678,315567,w,SK,H-W,...,37.0,4.0,6.0,0.0,TR,0.0,0.0,"[3477019, 3477020, 3477029, 3477016, 3477025, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
95,3555066,4062862960027,Chelsea Boot Rauleder blau,blau,56.451.36,6590678,315582,w,SK,H-W,...,39.0,6.0,10.0,11.0,TR,0.0,0.0,"[3555066, 3555070, 3555067, 3693948, 3693947, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
142,3474817,4062862677628,Elegante Stiefelette Glattleder blau,blau,55.510.26,6590677,315579,w,PT,H-W,...,42.5,8.5,15.0,15.0,Gummi,0.0,0.0,"[3474817, 3474816, 3851147, 3851144, 3474812, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
176,3711307,4062862408109,Sneaker low Glattleder schwarz,schwarz,56.305.57,6590678,315567,w,PT,H-W,...,44.0,9.5,17.0,0.0,TR,0.0,0.0,"[3711307, 3848838, 3848836, 3848841, 3848840, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
203,3692992,4062862487708,Sneaker high Rauleder schwarz,schwarz,0908.71.01,3692257,315586,w,PT,H-W,...,36.0,3.5,5.0,11.0,TR,0.0,0.0,"[3692992, 3692995, 3692998, 3788167, 3788154, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
216,3743199,4062862288329,Plateau Stiefelette Glattleder schwarz,schwarz,55.771.27,6590677,315595,w,PT,H-W,...,44.0,9.5,17.0,11.0,TPU,0.0,0.0,"[3743199, 3800405, 3788343, 3788347, 3743194, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
251,3693104,4062862092650,Schnürstiefelette Rauleder blau,blau,0873.90.11,3692257,315587,w,PT,H-W,...,42.0,8.0,14.0,14.0,TR,0.0,0.0,"[3693104, 3693106, 3843334, 3693107, 3693103, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
903,5206819,4062862537816,Slipper Rauleder grau,grau,64.213.39,6590677,315576,w,PT,F-S,...,40.0,6.5,11.0,0.0,TPU,0.0,0.0,"[5206819, 5206811, 5206816, 5206817, 5206814, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",4
964,5336280,4064032027240,Slipper Rauleder rosa,rosa,62.433.10,6590678,315576,w,PT,F-S,...,42.0,8.0,14.0,0.0,TR,0.0,0.0,"[5336280, 5336278, 5336281, 5336270, 5336283, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",8
1051,5568919,4062862615538,Pantolette Glattleder blau,blau,62.795.66,6590678,315572,w,PT,F-S,...,38.5,5.5,9.0,0.0,TPU,0.0,0.0,"[5568919, 5568923, 5568927, 5568920, 5568916, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
1585,7024066,4064032596029,Schnürstiefelette Rauleder rosa,rosa,72.776.41,6590678,315587,w,PT,H-W,...,42.5,8.5,15.0,17.0,TR,0.0,0.0,"[7024066, 7024060, 7024062, 7024057, 7024059, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",8


scores:  [0.05013624 0.03994945 0.03789608 0.03737419 0.03607996 0.03452116
 0.03418742 0.03366211 0.03318428 0.03219523]


In [17]:
user_id = next(iter(train.take(1)))['customer_id'].numpy()
predict_user(user_id)

predicting user:  9109803
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
57,3504613,4062862503521,Sneaker low Rauleder rot,rot,56.498.38,6590678,315567,w,PT,H-W,...,42.5,8.5,15.0,0.0,EVA,0.0,0.0,"[3504613, 3473297, 3504603, 3473298, 3473464, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",5
131,3481954,4062862072492,Slipper Glattleder braun,braun,55.231.24,6590677,315576,w,PT,H-W,...,40.0,6.5,11.0,0.0,EVA,0.0,0.0,"[3481954, 3591315, 3591319, 3591312, 3591313, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
134,3481043,4062862026716,Sneaker low Rauleder orange,orange,53.360.13,6590677,315567,w,SK,H-W,...,38.0,5.0,8.0,0.0,TR,0.0,0.0,"[3481043, 3481052, 3696792, 3481046, 3696791, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",9
361,3866345,4062862270997,Winterstiefelette Rauleder schwarz,schwarz,52.703.93,6590678,315580,w,PT,H-W,...,43.0,9.0,16.0,23.0,TR,0.0,0.0,"[3866345, 3866344, 4077515, 4077513, 4077517, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
584,3800494,4062862023166,Schnürstiefelette Rauleder,,52.865.41,6590678,315587,w,SK,H-W,...,43.0,9.0,16.0,12.0,TR,0.0,0.0,"[3800494, 3800495, 3481114, 6110643, 6249278, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",15
804,4847054,4062862569596,Slipper Glattleder weiß,weiß,64.213.21,6590677,315576,w,PT,F-S,...,37.0,4.0,6.0,0.0,TPU,0.0,0.0,"[4847054, 5150236, 5150525, 5135400, 5150234, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",2
1144,4821364,4062862534518,Sportliche Ballerina Rauleder Blau,Blau,64.111.16,6590677,315571,w,SK,F-S,...,44.0,9.5,17.0,0.0,TR,0.0,0.0,"[4821364, 4914057, 6282383, 6282390, 7596741, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 41.0, 42.0, 43....",18
1190,6406765,4064032694244,Mokassin Rauleder schwarz,schwarz,72.414.47,6590678,315590,w,PT,H-W,...,38.5,5.5,9.0,0.0,TR,0.0,0.0,"[6406765, 6406767, 6406766, 6406763, 6406764, ...","[35.5, 37.5, 38.0, 38.5, 39.0, 40.0, 40.5, 41....",0
3266,10434479,4065171881588,Slingpumps Materialmix Leder rosa,rosa,22.262.48,6590678,315565,w,SK,F-S,...,37.0,4.0,6.0,0.0,TPU,0.0,0.0,"[10434479, 10434481, 10434484, 10434482, 10434...","[35.0, 37.0, 38.0, 38.5, 40.0, 41.0, 43.0]",8


scores:  [0.05438135 0.04625572 0.04579283 0.04513082 0.04507356 0.04311597
 0.04245766 0.04052761 0.04046382 0.03971842]


In [18]:
#model.save('../../models/gabor/gabor_tensorflow_reco_all_data_3_epochs_v1.h5')